In [1]:
# necessary packages #

#using Pkg
#Pkg.add("Distances")
using Distributions
using Random
using Distances
using LinearAlgebra
using SparseArrays
using IterativeSolvers
using ProgressMeter
using JLD2
using DataFrames

In [2]:
include("../../../util2.j")

colnorm (generic function with 1 method)

In [3]:
# Packages for diagnostic test #
using StatsPlots
using MCMCChains
using PrettyTables

In [4]:
@load "../../data/conj_data/conj_data_expanded.jld"

┌ Warning: type OrderedCollections.OrderedDict{Symbol,Any} does not exist in workspace; reconstructing
└ @ JLD2 /home/lu/.julia/packages/JLD2/KjBIK/src/data.jl:1153


24-element Array{Symbol,1}:
 :U_indx      
 :S_indx      
 :N           
 :n           
 :m           
 :NN          
 :nnIndx_col  
 :nnIndx_row  
 :NM          
 :MnnIndxLU   
 :MnnIndx     
 :Mnndists    
 :coords      
 :coords_pred 
 :CVNN_train  
 :CVnnIndx_col
 :CVnnIndx_row
 :CVNN_test   
 :CVMnnIndxLU 
 :CVMnnIndx   
 :CVMnndists  
 :parts_train 
 :parts_test  
 :kf          

In [5]:
using RData
@time data_cleaned = load("../../data/rawdata/cleaned_data2_expanded.RData", convert = true)["data_cleaned2"];
@time hold_index_set = load("../../data/rawdata/hold_index_expanded.RData", convert = true);

 12.512621 seconds (32.06 M allocations: 2.904 GiB, 15.36% gc time)
  0.173269 seconds (49.89 k allocations: 15.984 MiB)


In [6]:
DataFrames.describe(data_cleaned)

,variable,mean,min,median,max
,Symbol,Float64,Float64,Float64,Float64
1,x,-1.04265e7,-1.11193e7,-1.03937e7,-1.00078e7
2,y,3.83719e6,3.33608e6,3.81747e6,4.44757e6
3,scaled_x,0.692742,0.0,0.725548,1.11149
4,scaled_y,0.501102,0.0,0.481382,1.11149
5,NDVI,0.240135,-0.00954541,0.192437,0.666598
6,EVI,0.15018,-0.00410843,0.122129,0.618208
7,red_reflectance,0.144423,0.0031,0.1452,0.3997
8,NIR_reflectance,0.243497,0.005,0.2383,0.6276
9,blue_reflectance,0.0752172,0.0001,0.0754,0.2


In [7]:
Y_ord = convert(Matrix, data_cleaned[:, [:NDVI, :red_reflectance]]);
X_ord = dropzeros(sparse([fill(1.0, (N, 1)) convert(Matrix, data_cleaned[:, 28:28])]));
p = 2; q = 2; K = 2;

In [8]:
data_cleaned = nothing; hold_index_set = nothing; GC.gc();

In [9]:
Sys.free_memory()/(2^20*1024)

2.212249755859375

In [10]:
# priors #
μβ = fill(0.0, p, q); inv_Vr =fill(0.0, p, p); # set Vr^{-1} be zero for the simulation...
νΣ = q + 1; ΨΣ = [[1.0 0.0]; [0.0 1.0]];

In [11]:
# using equation (12), we have
inv_Vstar = cholesky(X_ord[S_indx, :]'X_ord[S_indx, :] + inv_Vr)
μstar = inv_Vstar \ (X_ord[S_indx, :]'Y_ord[S_indx, :] + inv_Vr * μβ)
invUμstar = inv_Vstar.U * μstar;
Ψstar = ΨΣ + Y_ord[S_indx, :]'Y_ord[S_indx, :] + transpose(μβ) * inv_Vr * μβ -
    invUμstar'invUμstar;
νstar = νΣ + length(S_indx);
Lstar = cholesky(inv(inv_Vstar));

In [12]:
μstar

2×2 Array{Float64,2}:
  0.251445  0.139515 
 -0.133746  0.0602579

In [13]:
# Preallocation for posterior samples #
N_sam = 500;
β_sam = Array{Float64, 3}(undef, p, q, N_sam);
Σ_sam = Array{Float64, 3}(undef, q, q, N_sam);
Y_m_sam = Array{Float64, 3}(undef, NM, q, N_sam);   # missing value

cholΣ_sam = Array{Float64, 2}(undef, q, q);

In [14]:
# generate the posterior samples #
Random.seed!(123);
prog = Progress(N_sam, 1, "Computing initial pass...", 50)
for l in 1:N_sam
    Σ_sam[:, :, l] = rand(InverseWishart(νstar, Ψstar), 1)[1];    # sample Σ
    cholΣ_sam = cholesky(Σ_sam[:, :, l]);
    β_sam[:, :, l] = (Lstar.L * rand(Normal(), p, q)) * 
                    cholΣ_sam.U + μstar;          # sample γ  
    Y_m_sam[:, :, l] = X_ord[U_indx, :] * β_sam[:, :, l] + 
        rand(Normal(), NM, q) * cholΣ_sam.U;
    next!(prog) # monitor the progress
end

Computing initial pass...100%|██████████████████████████████████████████████████| Time: 0:00:05


In [15]:
N_Inf_burn = 1;
Y_m_pos_qt = Array{Float64, 3}(undef, NM, q, 3);
Y_m_pos_mean = Array{Float64, 2}(undef, NM, q);
Y_m_pos_var = Array{Float64, 2}(undef, NM, q);
for j in 1:q
    for i in 1:NM
        Y_m_pos_qt[i, j, :] = quantile(Y_m_sam[i, j, N_Inf_burn:N_sam], [0.025, 0.5, 0.975]);
        Y_m_pos_mean[i, j] = mean(Y_m_sam[i, j, N_Inf_burn:N_sam]);
        Y_m_pos_var[i, j] = var(Y_m_sam[i, j, N_Inf_burn:N_sam]);
    end
end

In [16]:
# count the covarage of 95% CI #
count = fill(0.0, 2);
for j in 1:q
    for i in 1:NM
        count[j] = count[j] + 
        ((Y_m_pos_qt[i, j, 1] < Y_ord[U_indx[i], j]) && 
            (Y_m_pos_qt[i, j, 3] > Y_ord[U_indx[i], j]))
    end
end
count

2-element Array{Float64,1}:
 64877.0
 66104.0

In [17]:
round.(count ./ NM, digits = 4)

2-element Array{Float64,1}:
 0.9664
 0.9847

In [18]:
round(sum(count) / (q * NM), digits = 4)

0.9755

In [19]:
# calculate root mean square predictive error #
MSPE1 = mean((Y_ord[U_indx, 1] - Y_m_pos_mean[:, 1]).^2)
RMSPE1 = sqrt(MSPE1);
MSPE2 = mean((Y_ord[U_indx, 2] - Y_m_pos_mean[:, 2]).^2)
RMSPE2 = sqrt(MSPE2); 
MSPE = mean((Y_ord[U_indx, :] - Y_m_pos_mean).^2)
RMSPE = sqrt(MSPE);
round.([RMSPE1 RMSPE2 RMSPE], digits = 5)

1×3 Array{Float64,2}:
 0.09899  0.04932  0.07821

In [20]:
# calculate root mean square predictive error #
MAE1 = mean(abs.(Y_ord[U_indx, 1] - Y_m_pos_mean[:, 1]))
MAE2 = mean(abs.(Y_ord[U_indx, 2] - Y_m_pos_mean[:, 2]))
MAE = mean(abs.(Y_ord[U_indx, :] - Y_m_pos_mean))
round.([MAE1 MAE2 MAE], digits = 5)

1×3 Array{Float64,2}:
 0.07563  0.0393  0.05746

In [21]:
# CRPS
CRPS = [(sqrt(Y_m_pos_var[i, j]) * ( 1 /sqrt(π) - 
        2 * pdf(Normal(), (Y_ord[U_indx[i], j] - Y_m_pos_mean[i, j]) / sqrt(Y_m_pos_var[i, j])) -
        ((Y_ord[U_indx[i], j] - Y_m_pos_mean[i, j]) / sqrt(Y_m_pos_var[i, j])) * 
        (2* cdf(Normal(), (Y_ord[U_indx[i], j] - Y_m_pos_mean[i, j]) / sqrt(Y_m_pos_var[i, j])) - 1 )))
        for i in 1:NM, j in 1:q];
round.([mean(CRPS[:, 1]) mean(CRPS[:, 2]) mean(CRPS)], digits = 5)

1×3 Array{Float64,2}:
 -0.05588  -0.02818  -0.04203

In [22]:
#INT
INT = [((Y_m_pos_qt[i, j, 3] - Y_m_pos_qt[i, j, 1]) + 
    (2 / 0.05)*(Y_m_pos_qt[i, j, 1] - Y_ord[U_indx[i], j]) * (Y_ord[U_indx[i], j] < Y_m_pos_qt[i, j, 1]) + 
    (2 / 0.05)*(Y_ord[U_indx[i], j] - Y_m_pos_qt[i, j, 3]) * (Y_ord[U_indx[i], j] > Y_m_pos_qt[i, j, 3]))
    for i in 1: NM, j in 1:q];
round.([mean(INT[:, 1]) mean(INT[:, 2]) mean(INT)], digits = 4)

1×3 Array{Float64,2}:
 0.5586  0.2437  0.4011

In [23]:
N_Inf_burn = 1;
summary_table = Array{Float64, 2}(undef, 7, 4);
summary_table[1, :] = vcat(mean(β_sam[1, 1, N_Inf_burn:N_sam]),
    quantile(β_sam[1, 1, N_Inf_burn:N_sam], [0.5, 0.025, 0.975]));
summary_table[2, :] = vcat(mean(β_sam[1, 2, N_Inf_burn:N_sam]),
    quantile(β_sam[1, 2, N_Inf_burn:N_sam], [0.5, 0.025, 0.975]));
summary_table[3, :] = vcat(mean(β_sam[2, 1, N_Inf_burn:N_sam]),
    quantile(β_sam[2, 1, N_Inf_burn:N_sam], [0.5, 0.025, 0.975]));
summary_table[4, :] = vcat(mean(β_sam[2, 2, N_Inf_burn:N_sam]),
    quantile(β_sam[2, 2, N_Inf_burn:N_sam], [0.5, 0.025, 0.975]));
summary_table[5, :] = vcat( mean(Σ_sam[1, 1, N_Inf_burn:N_sam]),
    quantile(Σ_sam[1, 1, N_Inf_burn:N_sam], [0.5, 0.025, 0.975]));
summary_table[6, :] = vcat(mean(Σ_sam[1, 2, N_Inf_burn:N_sam]),
    quantile(Σ_sam[1, 2, N_Inf_burn:N_sam], [0.5, 0.025, 0.975]));
summary_table[7, :] = vcat( mean(Σ_sam[2, 2, N_Inf_burn:N_sam]),
    quantile(Σ_sam[2, 2, N_Inf_burn:N_sam], [0.5, 0.025, 0.975]));
summary_table = round.(summary_table; digits = 6);
rnames = ["β[1, 1]", "β[1, 2]", "β[2, 1]", "β[2, 2]", "Σ[1, 1]", "Σ[1, 2]", "Σ[2, 2]"];
summary_table = [rnames summary_table];
pretty_table(summary_table,  ["" "mean" "median" "2.5%" "97.5%"], markdown)

|         |      mean |    median |      2.5% |     97.5% |
|---------|-----------|-----------|-----------|-----------|
| β[1, 1] |  0.251445 |  0.251445 |  0.251307 |  0.251582 |
| β[1, 2] |  0.139514 |  0.139515 |  0.139444 |  0.139583 |
| β[2, 1] | -0.133746 | -0.133735 | -0.134247 | -0.133285 |
| β[2, 2] |  0.060255 |  0.060249 |  0.060024 |  0.060517 |
| Σ[1, 1] |   0.01599 |   0.01599 |  0.015965 |  0.016017 |
| Σ[1, 2] | -0.006494 | -0.006494 | -0.006505 | -0.006483 |
| Σ[2, 2] |  0.003656 |  0.003656 |  0.003651 |  0.003662 |
